# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [111]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import mykey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [112]:
cities = os.path.join( '..','WeatherPy/output', 'weather_data.csv')

cities_df = pd.read_csv(cities, encoding="ISO-8859-1")
cities_df.head()


,City ID,city,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed,country,Date
0,0,half moon bay,56.93,37.4636,-122.4286,84,75,13.80,US,1655119843
1,1,ribeira grande,66.60,38.5167,-28.7000,82,40,16.11,PT,1655119764
2,2,husavik,53.55,66.0449,-17.3389,70,100,5.73,IS,1655120258
3,3,nikolskoye,68.88,59.7035,30.7861,48,48,7.02,RU,1655119773
4,4,aswan,106.90,24.0934,32.9070,13,0,10.36,EG,1655120260


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [113]:
# Create a map using state centroid coordinates to set markers
marker_locations = cities_df[['Lat', 'Lng']]
humidity = cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [114]:
ideal_df = cities_df.loc[ (cities_df["Max Temp"]<80) 
                        & (cities_df["Max Temp"]>70) 
                        & (cities_df["Wind Speed"]<10) 
                        & (cities_df["Cloudiness"]==0)]

ideal_df = ideal_df.reset_index()  
ideal_df = ideal_df.dropna()  # There are cities without country information in the dataset, I dropped that data from this list.
print(ideal_df)

    index  City ID            city  Max Temp      Lat       Lng  Humidity  \
0      12       12            hilo     77.29  19.7297 -155.0900        85   
1      56       56     fort morgan     71.02  40.2503 -103.8000        48   
2     170      170       opelousas     78.69  30.5335  -92.0815       100   
3     191      191        marshall     78.39  34.3834  -86.3333        92   
4     267      267  port elizabeth     73.31 -33.9180   25.5701        67   
6     335      335        karratha     75.00 -20.7377  116.8463        65   
7     337      337          beloha     79.81 -25.1667   45.0500        23   
8     384      384       amboasary     78.73 -25.0333   46.3833        37   
10    485      485       roebourne     75.02 -20.7833  117.1333        63   

    Cloudiness  Wind Speed country        Date  
0            0        8.05      US  1655119772  
1            0        8.05      US  1655120299  
2            0        4.61      US  1655120388  
3            0        0.00      U

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [115]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
target_radius = 5000
target_search = "hotel"

insert_control = False
d_hotel_list  = []
d_city_list   = []
d_lat_list    = []
d_long_list   = []
d_county_list = []


for index, row in ideal_df.iterrows():    
    target_coordinates  = f"{row['Lat']},{row['Lng']}"


    insert_control = False
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,        
        "radius": target_radius,
        "keyword": target_search,
        "maxResults":1,
        #"type": target_type,
        "key": mykey
    }


    try:
        response = requests.get(base_url, params=params)   
        places_data = response.json()
        
   
        if places_data['status'] == "ZERO_RESULTS":
            pass
        elif (places_data['status'] != "ZERO_RESULTS") & (insert_control == False):
 
            d_hotel_list.append(places_data["results"][0]["name"])
            d_city_list.append(row['city'])
            d_county_list.append(row["country"])
            d_lat_list.append(places_data["results"][0]["geometry"]["location"]["lat"])
            d_long_list.append(places_data["results"][0]["geometry"]["location"]["lng"])
            insert_control = True

    except KeyError:
        print(f"Unexpected Error during building listing") 

    # run a request using our params dictionary
    

    # convert response to json
   

print(d_city_list)
print(d_county_list)
print(d_hotel_list)
print(d_lat_list)
print(d_long_list)
    

    

['hilo', 'fort morgan', 'opelousas', 'marshall', 'port elizabeth', 'karratha', 'amboasary', 'roebourne']
['US', 'US', 'US', 'US', 'ZA', 'AU', 'MG', 'AU']
['Hilo Hawaiian Hotel', 'Hampton Inn Fort Morgan', 'Evangeline Downs Hotel, Ascend Hotel Collection', 'Lake Guntersville Bed & Breakfast', 'Herms Restaurant and Boutique Hotel', 'Karratha International Hotel', 'DISCOVERY AMBOASARY', 'Latitude20 Roebourne Village']
[19.7283446, 40.26080390000001, 30.5268478, 34.3623906, -33.9486733, -20.7376505, -25.039133, -20.7544272]
[-155.0664264, -103.8005018, -92.0606842, -86.2930674, 25.569334, 116.8417759, 46.38823, 117.1456981]


In [116]:
hotel_df = pd.DataFrame({            
            'City':d_city_list,            
            'Lat':d_lat_list,
            'Lng':d_long_list,             
            'Hotel Name': d_hotel_list,
            'Country':d_county_list
        })

pprint(hotel_df)

             City        Lat         Lng  \
0            hilo  19.728345 -155.066426   
1     fort morgan  40.260804 -103.800502   
2       opelousas  30.526848  -92.060684   
3        marshall  34.362391  -86.293067   
4  port elizabeth -33.948673   25.569334   
5        karratha -20.737651  116.841776   
6       amboasary -25.039133   46.388230   
7       roebourne -20.754427  117.145698   

                                        Hotel Name Country  
0                              Hilo Hawaiian Hotel      US  
1                          Hampton Inn Fort Morgan      US  
2  Evangeline Downs Hotel, Ascend Hotel Collection      US  
3                Lake Guntersville Bed & Breakfast      US  
4              Herms Restaurant and Boutique Hotel      ZA  
5                     Karratha International Hotel      AU  
6                              DISCOVERY AMBOASARY      MG  
7                     Latitude20 Roebourne Village      AU  


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [119]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))